<a href="https://colab.research.google.com/github/Vamarinpe/PAE/blob/master/CodRecurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Por agente

# importa la librería
import pandas as pd
import numpy as np
pd.set_option('display.notebook_repr_html', False)
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import csv

Datos = pd.read_csv(
    "https://raw.githubusercontent.com/Vamarinpe/PAE/master/Recurso.csv",
    sep = ',',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Fecha","Codigo Agente","Recurso"],
    usecols=["Fecha","Codigo Agente","Recurso","Precio Oferta Ideal"],      # Datosnuevo=Datos[['Fecha','Codigo Agente','Recurso']]  - otra forma
    parse_dates=["Fecha"],
    dayfirst=True)


#Datos = pd.read_csv(
    #"https://raw.githubusercontent.com/Vamarinpe/Ensayo2/master/Datos.csv",
    #sep = ',',         # separador de campos
    #thousands = None,  # separador de miles para números
    #decimal = '.',
    #quoting=csv.QUOTE_NONNUMERIC)

#Datos['Fecha'] = Datos.Fecha\
                             #.map(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [0]:
len(Datos)
Datos.columns
Datos.describe()
Datos.head(60)

In [0]:
Datos.count()

In [0]:
Datos.groupby('Codigo Agente')['Precio Oferta Ideal'].count()
Datos.groupby('Recurso')['Precio Oferta Ideal'].count()
Datos.groupby('Fecha')['Precio Oferta Ideal'].count()            # también sin el ['Precio Oferta Ideal']
#Datos.groupby('Codigo Agente').count()

In [0]:
Datos.loc[('2012-09-04')]
Datos.loc[('2012-09-04','EPMG')]
plt.plot(Datos.loc[('2017-09-05','EPMG')])
plt.plot(Datos.loc[('2017-09-06','EPMG')])
plt.plot(Datos.loc[('2019-08-31','EPMG')])
plt.show()

In [0]:
Datos.loc['2012-01-01':'2013-12-31']

In [0]:
Datos.xs('2012-06-08',level='Fecha')
Datos.xs('EPMG',level='Codigo Agente')
Datos.xs('GUATAPE',level='Recurso').plot()
Datos.xs('PLAYAS',level='Recurso').plot()
plt.rcParams["figure.figsize"] = (20,8)


In [40]:
Datos.iloc[5]

Precio Oferta Ideal    135.66
Name: (2000-01-01 00:00:00, ESSG, BARRANCA 4), dtype: float64

In [98]:
def capacidad(x):
  Cap = pd.read_csv(
    x,
    sep = ';',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Recurso"],
    usecols=["Recurso","Capacidad Efectiva Neta kW","Combustible por defecto"])
  return Cap.sort_values(by='Capacidad Efectiva Neta kW',ascending=False).drop_duplicates().head(10)

C2000=capacidad("/content/Capacidad_Efectiva_Neta_(kW)_2000.csv")
C2001=capacidad("/content/Capacidad_Efectiva_Neta_(kW)_2001.csv")

C2000

           Combustible por defecto  Capacidad Efectiva Neta kW
Recurso                                                       
SAN CARLOS                    AGUA                   1240000.0
GUAVIO                        AGUA                   1150000.0
TEBSAB                         GAS                    750000.0
CHIVOR                        AGUA                    750000.0
PAGUA                         AGUA                    581000.0
GUATAPE                       AGUA                    560000.0
CASALCO                       AGUA                    543000.0
BETANIA                       AGUA                    540000.0
GUATRON                       AGUA                    512000.0
ALBAN                         AGUA                    439000.0

In [100]:
C2000.groupby('Combustible por defecto').count()

                         Capacidad Efectiva Neta kW
Combustible por defecto                            
AGUA                                              9
GAS                                               1

In [108]:
#Datos.xs(C2000.index[4],level='Recurso')
Datos[Datos.index.get_level_values('Recurso').isin(list(C2000.index))]


                                     Precio Oferta Ideal
Fecha      Codigo Agente Recurso                        
2000-01-01 CHBG          BETANIA                   44.95
           EMGG          CASALCO                   49.70
           CHVG          CHIVOR                    48.37
           EPMG          GUATAPE                   25.59
                         GUATRON                   58.28
           EMGG          GUAVIO                    37.92
                         PAGUA                     39.03
           ISGG          SAN CARLOS                25.59
           CRLG          TEBSAB                    68.59
2000-01-02 CHBG          BETANIA                   44.28
           EMGG          CASALCO                   49.70
           CHVG          CHIVOR                    50.95
           EPMG          GUATAPE                   25.59
                         GUATRON                   57.28
           EMGG          GUAVIO                    38.95
                         PAGUA 